# Importing data and basic infos

# Spaceship Classification

**we will try to cpredict if a person is transported to an other dimension**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix ,  plot_roc_curve , classification_report , accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv("../input/spaceship-titanic/train.csv")
df_test=pd.read_csv("../input/spaceship-titanic/test.csv")

In [ ]:
print("basic shape info train  {}".format(df_train.shape))
print("basic shape info test  {}".format(df_test.shape))

In [ ]:
df_train.head(8)

In [ ]:
df_test.head()

**PassengerId**     unique identifier

**HomePlanet**     The planet the passenger departed from

**CryoSleep**     Indicates whether the passenger elected to be put into suspended animation for the voyage.

**Cabin**     deck/num/side of cabin side can be P for port or S for starboard

**Destination** The planet the passenger will be debarking to.

**Age**     ..

**VIP**     Vip service

**RoomService** amount billed by passenger

**FoodCourt**     //

**ShoppingMall** //

**Spa** //

**VRDeck**// 

**Name**   

**Transported** Whether the passenger was transported to another dimension.


In [ ]:
df_train.describe()

The mean age of the passengers is 28.9 years 

# Cleaning Data

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

Dropping names because there is no information in it


# Cleaning Data

In [ ]:
df_train=df_train.drop(["Name"],axis=1)
df_test=df_test.drop(["Name"],axis=1)

In [ ]:
#replacing numerical data
df_train["RoomService"].fillna(df_train["RoomService"].mean(),inplace=True)
df_train["Age"].fillna(df_train["Age"].mean(),inplace=True)
df_train["FoodCourt"].fillna(df_train["FoodCourt"].mean(),inplace=True)
df_train["ShoppingMall"].fillna(df_train["ShoppingMall"].mean(),inplace=True)
df_train["Spa"].fillna(df_train["Spa"].mean(),inplace=True)
df_train["VRDeck"].fillna(df_train["VRDeck"].mean(),inplace=True)

df_test["RoomService"].fillna(df_test["RoomService"].mean(),inplace=True)
df_test["Age"].fillna(df_test["Age"].mean(),inplace=True)
df_test["FoodCourt"].fillna(df_test["FoodCourt"].mean(),inplace=True)
df_test["ShoppingMall"].fillna(df_test["ShoppingMall"].mean(),inplace=True)
df_test["Spa"].fillna(df_test["Spa"].mean(),inplace=True)
df_test["VRDeck"].fillna(df_test["VRDeck"].mean(),inplace=True)


#replacing categorical data
df_train['VIP'].fillna(df_train['VIP'].value_counts().idxmax(),inplace=True)
df_train['HomePlanet'].fillna(df_train['HomePlanet'].value_counts().idxmax(),inplace=True)
df_train['CryoSleep'].fillna(df_train['CryoSleep'].value_counts().idxmax(),inplace=True)
df_train['Cabin'].fillna(df_train['Cabin'].value_counts().idxmax(),inplace=True)
df_train['Destination'].fillna(df_train['Destination'].value_counts().idxmax(),inplace=True)

df_test['VIP'].fillna(df_test['VIP'].value_counts().idxmax(),inplace=True)
df_test['HomePlanet'].fillna(df_test['HomePlanet'].value_counts().idxmax(),inplace=True)
df_test['CryoSleep'].fillna(df_test['CryoSleep'].value_counts().idxmax(),inplace=True)
df_test['Cabin'].fillna(df_test['Cabin'].value_counts().idxmax(),inplace=True)
df_test['Destination'].fillna(df_test['Destination'].value_counts().idxmax(),inplace=True)

replacing data with mean values and most frequent

In [ ]:
df_train.isnull().sum()
df_test.isnull().sum()

# Checking outliers with isolation forest

In [ ]:
df_train.dtypes

In [ ]:

from sklearn.ensemble import IsolationForest


def detect_outliers(dataframe,contamination):
    

    a=list(dataframe.select_dtypes(['int64']).columns)+list(dataframe.select_dtypes(['float64']).columns)
    model=IsolationForest(contamination=contamination)
    model.fit(dataframe[a])
    outliers = model.predict(dataframe[a]) ==-1
    return outliers



In [ ]:
contamination=0.005
index=detect_outliers(df_train,contamination)
df_train[index].head()

In [ ]:
df_train=df_train[index==False]



# Countplots

In [ ]:
from math import ceil
sns.set_style('darkgrid')
def countplot(df,max_columns,val,figsize,title_size=20,title="",fontscale=2):
    sns.set(font_scale = fontscale)
    l=len(df.columns)
    ligns=ceil(l/max_columns)
    fig=plt.figure(1,figsize=figsize)
    i=1
    
    for col in df.columns:
        if len(pd.unique(df[col]))<val:
            plt.subplot(ligns,max_columns,i)
            plt.title(col, fontsize=title_size)
            plt.grid()
            i=i+1
            sns.countplot(data = df, x=col )
    fig.tight_layout(pad=3.0)
    
    fig.suptitle(title)
    plt.show()
    
    
#countsplot dataframe,columns spliting,max_uniquevalues,figsize,title,title_size
    
title="Different Countplots"
countplot(df_train,2,10,(100,200),80,fontscale=7)
#countsplot dataframe,columns spliting,max_uniquevalues,figsize,title,title_size,fontscale

Countplots:

-We can see than most people come from earth

-the majority don't want cryosleep

-most of the people go for "Trappist-1e"

-most of te people don't go VIP

-the transported category is balanced


Visualizing each category of cabin:

# Continous variables

In [ ]:
#Age 	RoomService 	FoodCourt 	ShoppingMall 	Spa 	VRDeck
figure2=plt.figure(2,figsize=(25,20))
sns.set()
plt.subplot(3,2,1)
sns.violinplot(data=df_train,x="RoomService")
plt.subplot(3,2,2)
sns.violinplot(data=df_train,x="Age")
plt.subplot(3,2,3)
sns.violinplot(data=df_train,x="FoodCourt")
plt.subplot(3,2,4)
sns.violinplot(data =df_train, x="ShoppingMall")
plt.subplot(3,2,5)
sns.violinplot(data=df_train,x="Spa")
plt.subplot(3,2,6)
sns.violinplot(data=df_train,x="VRDeck")

plt.suptitle("Repartition of Roomservice,Age,FoodCourt,ShoppingMall,Spa,VRDeck")


figure2.tight_layout(pad=3.0)


plt.show()

# Mean deviations

In [ ]:
moy= df_train["Transported"].mean()
figclass=plt.figure(78,figsize=(18,14))
figclass.suptitle('mean deviations of target mean')
sns.set()
plt.subplot(3,3,1)
plt.title("Destination")

(df_train.groupby('Destination')["Transported"].mean()-moy).plot(kind='bar',color='Green',ls='dashed',edgecolor='Black')
plt.axhline(y=0)
plt.subplot(3,3,2)

plt.title("CryoSleep")
(df_train.groupby('CryoSleep')["Transported"].mean()-moy).plot(kind='bar',color='Blue',ls='dashed',edgecolor='Black')
plt.axhline(y=0)

plt.subplot(3,3,3)
plt.title("HomePlanet")
(df_train.groupby('HomePlanet')["Transported"].mean()-moy).plot(kind='bar',color='Red',ls='dashed',edgecolor='Black')
plt.axhline(y=0)

plt.subplot(3,3,4)
plt.title("VIP")
(df_train.groupby('VIP')["Transported"].mean()-moy).plot(kind='bar',color='Red',ls='dashed',edgecolor='Black')
plt.axhline(y=0)

figclass.tight_layout(pad=3.0)


# Pairplots

In [ ]:
sns.pairplot(df_train[list(df_train.select_dtypes(['int64']).columns)+list(df_train.select_dtypes(['float64']).columns)])

# Parsing Cabin

In [ ]:
"Cabin: deck/num/side"

sp0=lambda x: x.split("/")[0]
sp1=lambda x: x.split("/")[1]
sp2=lambda x: x.split("/")[2]

deck=df_train.Cabin.apply(sp0)
num=df_train.Cabin.apply(sp1)
side=df_train.Cabin.apply(sp2)
data={'deck':deck,'side':side}
df_cabin=pd.DataFrame(data)

countplot(df_cabin,2,10,(10,5),12,fontscale=1)

Hetamap correlations of numerical values

# Correlations

In [ ]:
figsns=plt.figure(figsize=(25,25))

sns.heatmap(df_train.iloc[1:].corr(),annot=True)

plt.show()

we can see cryosleep is highly correlated to the target Spa RoomService VRDeck too, VIP and Age less

Cryosleep is correlated with Shoppingmall spa VRdeck FoodCourt

We can check correlations of the different cabin characteristics "side" and "deck"

In [ ]:
df_cabin["Transported"]=df_train["Transported"]
figZ=plt.figure(figsize=(20,20))

sns.heatmap(pd.get_dummies(df_cabin).corr(),annot=True)
plt.show()

deck_test=df_train.Cabin.apply(sp0)
side_test=df_train.Cabin.apply(sp2)
data={'deck':deck,'side':side}
df_cabin_test=pd.DataFrame(data)


df_test["side"]=df_cabin_test["side"]
df_test["deck"]=df_cabin_test["deck"]


df_train["side"]=df_cabin["side"]
df_train["deck"]=df_cabin["deck"]

df_train=df_train.drop(["Cabin"],axis=1)
df_test=df_test.drop(["Cabin"],axis=1)

We can try to combine RoomService, FoodCourt, ShoppingMall, Spa, VRDeck wich are all luxury amenities

In [ ]:

df_train["Luxury"]=df_train["RoomService"]+df_train["FoodCourt"]+df_train["ShoppingMall"]+df_train["Spa"]+df_train["VRDeck"]
df_test["Luxury"]=df_test["RoomService"]+df_test["FoodCourt"]+df_test["ShoppingMall"]+df_test["Spa"]+df_test["VRDeck"]

In [ ]:
figsns2=plt.figure(figsize=(25,25))

sns.heatmap(df_train.iloc[1:].corr(),annot=True)

plt.show()

# PCA 2 and 3 components:

In [ ]:
#two principal components

pca = PCA(n_components=2)
principalComponents=pca.fit_transform(pd.get_dummies(df_train))

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df_train[['Transported']]], axis = 1)

In [ ]:
total_var = pca.explained_variance_ratio_.sum() * 100

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = df_train["Transported"].values


colors = ['r', 'g']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Transported'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

total_var = pca.explained_variance_ratio_.sum() * 100
print("total variance {:.2f}  \n  explained_var_ratio{}".format(total_var,pca.explained_variance_ratio_))

In [ ]:
X=df_train.drop(["Transported"],axis=1)
X=pd.get_dummies(X)
pca = PCA(n_components=3)
components = pca.fit_transform(X)


fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df_train['Transported'],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()
print("explained_var_ratio{}".format(pca.explained_variance_ratio_))

We can see that we can "correclty" separate two classes with two and three components

Set up train and test data

In [ ]:
X_train=pd.get_dummies(df_train.drop(["PassengerId","Transported"],axis=1))
X_test=pd.get_dummies(df_test.drop(["PassengerId"],axis=1))
y_train=df_train["Transported"]

Let's try a first simple multi layer perceptron with pca choices of features

# Feature selection

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

# Fit the classifier with max_depth=3
clf = DecisionTreeClassifier(max_depth=3, random_state=1234)
model = clf.fit(X_train, y_train)




#text rep
text_representation = tree.export_text(clf,feature_names=list(X_train.columns))
print(text_representation)



## PCA selector

In [ ]:
pca = PCA(.85)
#minimum number of components so that 85% of the variance is retained
pca.fit(X_train)
print("number of component retained {}".format(pca.n_components_))


X_train_pca=pca.transform(X_train)
X_test_pca=pca.transform(X_test)

In [ ]:
#-----------------------MLP grid search CV-----------------------------
mlp=make_pipeline(RobustScaler(),MLPClassifier(max_iter=10000))
param_grid={
            'mlpclassifier__hidden_layer_sizes':(range(1,4)),
            'mlpclassifier__solver':['lbfgs','sgd'],
            'mlpclassifier__activation':["logistic", "tanh", "relu"]
           }                             
grid= GridSearchCV(mlp,param_grid=param_grid,cv=5)
grid.fit(X_train_pca,y_train)
print("best train cv score : {:.3f}, with parameters : {}".format(grid.best_score_,grid.best_params_))
mlp=grid.best_estimator_

In [ ]:
#--------------------RandomForest--------------------
rdm=make_pipeline(RandomForestClassifier())

param_rdm = {
             'randomforestclassifier__n_estimators': [150,170,190],
             'randomforestclassifier__max_features': ['auto'],
             'randomforestclassifier__max_depth' : [3,4,5],
             'randomforestclassifier__criterion' :['gini'],
            }

grid_rdm=GridSearchCV(rdm,param_rdm,cv=5)
grid_rdm.fit(X_train_pca,y_train)
rdm=grid_rdm.best_estimator_
print("best score rdm {:.3f} aw parameters :{}".format(grid_rdm.best_score_,grid_rdm.best_params_))

# Feature selection using chi2 test

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
X_ktrain=[]
k_cv_scores=[]
k_test_scores=[]
for n in range(1,7):
    selector=SelectKBest(chi2,k=n)
    selector.fit_transform(X_train,y_train)
    best=list(X_train.columns[selector.get_support()])
    X_ktrain.append(X_train[best])

#--------------------RandomForest--------------------
rdm=make_pipeline(RandomForestClassifier())

param_rdm = {
             'randomforestclassifier__n_estimators': [170,180,190,200,210,220],
             'randomforestclassifier__max_features': ['auto'],
             'randomforestclassifier__max_depth' : [3,4,5,6,7,8],
             'randomforestclassifier__criterion' :['gini'],
            }

grid_rdm=GridSearchCV(rdm,param_rdm,cv=5)



for Z in X_ktrain:
    grid_rdm.fit(Z,y_train)
    test=X_test[list(Z.columns)]
    print("best train cv score : {:.3f}  with parameters {}".format(grid_rdm.best_score_,grid_rdm.best_params_))
    k_cv_scores.append(grid_rdm.best_score_)



figkbest=plt.figure(4,figsize=(8,8))


plt.plot(range(1,7),k_cv_scores,'r-')
plt.title("selectkbest")


plt.show()

In [ ]:
max_value = max(k_cv_scores)
max_index = k_cv_scores.index(max_value)

print("max value {:.3f} at index  {}".format(max_value,max_index))




# Conclusion of feature selection

In [ ]:

X_ktrain[5]



In [ ]:
#--------------------RandomForest--------------------
rdm=make_pipeline(RobustScaler(),RandomForestClassifier())

param_rdm = {
             'randomforestclassifier__n_estimators': [170,180,190,200,210,220],
             'randomforestclassifier__max_features': ['auto'],
             'randomforestclassifier__max_depth' : [3,4,5,6,7,8],
             'randomforestclassifier__criterion' :['gini'],
            }

grid_rdm=GridSearchCV(rdm,param_rdm,cv=5)
grid_rdm.fit(X_ktrain[5],y_train)
best_rdm=grid_rdm.best_estimator_